# ノードの再開

---

CoursewareHub環境の停止したノードを再開します。

> このNotebookは AWS, AzureのVCノードに対してのみ利用可能です。他のクラウドプロバイダで実行中のVCノードはノード再開の機能をVCPがサポートしていないためエラーとなります。

## パラメータの指定

ノードを再開するのに必要となるパラメータを入力します。

### UnitGroup名

操作対象となるVCPのUnitGroup名を指定します。

VCノードを作成時に指定したUnitGroup名を確認するために `group_vars` ファイル名の一覧を表示します。

In [ ]:
!ls -1 --hide all group_vars/

UnitGroup名を指定してください。

In [ ]:
# (例)
# ugroup_name = 'CoursewareHub'

ugroup_name = 

UnitGroupの変数をgroup_varsファイルから読み込みます。

In [ ]:
%run scripts/group.py
gvars = load_group_vars(ugroup_name)

### VCCアクセストークンの入力

VCノードを停止するためにVC Controller(VCC)のアクセストークンが必要となります。
次のセルを実行すると表示される入力枠にVCCのアクセストークンを入力してください。

> アクセストークン入力後に Enter キーを押すことで入力が完了します。

In [ ]:
from getpass import getpass
vcc_access_token = getpass()

入力されたアクセストークンが正しいことを、実際にVCCにアクセスして確認します。

In [ ]:
from common import logsetting
from vcpsdk.vcpsdk import VcpSDK

vcp = VcpSDK(vcc_access_token)

上のセルの実行結果がエラーとなり以下のようなメッセージが表示されている場合は、入力されたアクセストークンに誤りがあります。

```
config vc failed: http_status(403)
2023/XX/XX XX:XX:XX UTC: VCPAuthException: xxxxxxx:token lookup is failed: permission denied
```

エラーになった場合はこの節のセルを全て `unfreeze` してから、もう一度アクセストークンの入力を行ってください。

## ノードの再開

現在のノードの状態を確認します。

In [ ]:
ug = vcp.get_ugroup(ugroup_name)
ug.df_nodes()

### managerノードの再開

managerノードを再開します。

> managerノードを停止していない場合は、この節をスキップしてください。

In [ ]:
unit_manager = ug.get_unit('manager')
unit_manager.power_on_nodes()

managerノードの状態を確認します。`node_state` の表示が `RUNNING`となっていることを確認してください。

In [ ]:
unit_manager.df_nodes()

managerノードをNFSサーバとして利用している場合はNFSサーバが起動していることを確認します。

In [ ]:
if 'nfs_ipaddress' not in gvars:
    !ansible {ugroup_name}_manager -b -a 'systemctl status nfs-server'

上のセルがエラーとなり `Active: inactive (dead)` と表示されている場合はNFSサーバが起動していません。
そのような場合は次のセルのコメントを外して実行し、手動で起動できるかを確認してください。

In [ ]:
# !ansible {ugroup_name}_manager -b -a 'systemctl start nfs-server'

サービスを手動で起動した場合は、再度NFSサーバサービスの状態を確認します。次のセルのコメントを外して実行してください。

In [ ]:
# !ansible {ugroup_name}_manager -b -a 'systemctl status nfs-server'

### workerノードの再開

workerノードを再開します。

In [ ]:
unit_worker = ug.get_unit('worker')
unit_worker.power_on_nodes(num_power_on_nodes=gvars['worker_nodes'])

以下のようなメッセージが表示されエラーとなった場合は、下のセルでノードの状態を確認してください。
```
2023-XX-XX XX:XX:XX,XXX - ERROR - 2023/XX/XX XX:XX:XX UTC: VCPStateConflictException: xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx:worker:| Conflict: unit is now processing, so cannot apply
```

workerノードの状態を確認します。全ノードの `node_state` の表示が `RUNNING`となっていることを確認してください。

In [ ]:
unit_worker.df_nodes()

一部のVCノードが `POWER_OFF`のままの場合は、次のセルのコメントを外して実行してください。その際に対象とするVCノードの`node_no`の値を指定してください。`node_no`の値は上のセルの出力結果で確認してください。

In [ ]:
# node_no =
# unit_worker.power_on_nodes(node_no=node_no)
# unit_worker.df_nodes()

スケジュール設定が行われている場合はsystemdタイマーを再開します。

In [ ]:
if any(name in gvars for name in ["vcnode_schedule", "schedule"]):
    !ansible {ugroup_name}_manager -b -m systemd \
        -a 'name=cwh-worker-node.timer enabled=true state=started'

## 状態の確認

docker swarmで実行しているサービスの状態を確認します。

In [ ]:
!ansible {ugroup_name}_manager -a 'docker stack services coursewarehub'

In [ ]:
!ansible {ugroup_name}_manager -a 'docker stack ps coursewarehub'

しばらく待つとコンテナは自動的に起動します。ただしコンテナが起動してもCoursewareHubが完全に起動するまでは、ログインに成功してもエラーとなる場合があります。起動してから５分程度は起動が完了するまで待ってください。

５分以上待っても状態がおかしい場合は起動しているコンテナをいったんすべて削除した後に再配備したほうが良いことがあります。以下のセルのコメントを外して実行してください。

In [ ]:
# コンテナの削除
# !ansible {ugroup_name}_manager -a 'docker stack rm coursewarehub'

In [ ]:
# コンテナの再配備
# !ansible {ugroup_name}_manager -a 'chdir={{{{compose_dir}}}} \
#    docker stack deploy -c docker-compose.yml coursewarehub'